# Test notebook to exercise the Lasair API

In [1]:
#!/usr/bin/pip3 install lasair --upgrade
import sys

Change this to the API token and endpoint you want to test

In [2]:
lsst=True # if you change this restart kernel

if lsst:
    sys.path.append('API_lsst')
    import settings
    endpoint = "https://lasair-lsst.lsst.ac.uk/api"
    oid = 'diaObjectId'
    raname = 'ra'
    decname = 'decl'
else:
    sys.path.append('API_ztf')
    import settings
    endpoint = "https://lasair-ztf.lsst.ac.uk/api"
    oid = 'objectId'
    raname = 'ramean'
    decname = 'decmean'

Fetch your API TOKEN from your Lasair account, top right under "profile"

In [3]:
import json
from lasair import LasairError, lasair_client as lasair
import settings
L = lasair(settings.API_TOKEN, endpoint=endpoint)

/Users/rwillia5/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


This function simply prints the first lines of the given string.
You can replace it with the normal print if you want to see everything.

In [4]:
def print15(s):
    print('\n'.join(s.split('\n')[:15]))

## Cone search

In [5]:
racone  = 279.22
deccone = -47.91
radius = 60 # arcsec
try:
    result = L.cone(racone, deccone, radius, requestType='nearest')
    print(json.dumps(result, indent=2))
except:
    print('nothing found')

{}


## Database query

In [6]:
selected = 'objects.%s, objects.%s, objects.%s' % (oid, raname, decname)
tables = 'objects,sherlock_classifications'
conditions = 'classification="SN"'
results = L.query(selected, tables, conditions, limit = 8)

for row in results:
    objectId = row[oid]
    ra = row[raname]
    dec = row[decname]
    print(objectId, ra, dec)

# Just use this objectId in subsequent calls
objectId = str(objectId)

3516505585459658821 54.869760771850316 -27.294073926963623
3516505612303204354 51.358273330001936 -27.692442189066135
3516506109982539799 52.84428812866385 -26.567196010769738
3516506171722694675 51.96725530812655 -28.4613174524388
3516508308468924426 53.01968624151246 -28.292557085291445
3516508308468924434 53.085036036883515 -28.37399123521993
3521874256998694923 54.99875627554984 -27.308598190156637
3521874258609307650 54.82056823745111 -27.176290651327594


## Lightcurves

In [7]:
result = L.object(objectId, lasair_added=False)
print15(json.dumps(result, indent=2))

{
  "diaObjectId": "3521874258609307650",
  "diaSources": [
    {
      "diaSourceId": 3527246281296576548,
      "midpointMjdTai": 61314.27337522106,
      "band": "g",
      "psfFlux": 4045.029296875,
      "psfFluxErr": 961.5590209960938
    },
    {
      "diaSourceId": 3527246818167488548,
      "midpointMjdTai": 61314.27337522106,
      "band": "g",
      "psfFlux": 4044.686279296875,


## Sherlock Object

In [8]:
lite=True
result = L.sherlock_object(objectId, lite=lite)
print15(json.dumps(result, indent=2))

{
  "classifications": {
    "3521874258609307650": [
      "SN",
      "The transient is possibly associated with <em>03391748-2710138</em>; a J=14.72 mag galaxy found in the 2MASS catalogue. It's located 20.76\" S, 7.30\" W from the galaxy centre."
    ]
  },
  "crossmatches": [
    {
      "Mag": 14.716,
      "MagErr": 0.13,
      "MagFilter": "J",
      "association_type": "SN",
      "catalogue_object_id": "03391748-2710138",
      "catalogue_object_type": "galaxy",


## Sherlock Position

In [9]:
result = L.sherlock_position(ra, dec, lite=lite)
print15(json.dumps(result, indent=2))

{
  "classifications": {
    "query0": [
      "SN",
      "The transient is possibly associated with <em>03391748-2710138</em>; a J=14.72 mag galaxy found in the 2MASS catalogue. It's located 20.76\" S, 7.30\" W from the galaxy centre."
    ]
  },
  "crossmatches": [
    {
      "Mag": 14.716,
      "MagErr": 0.13,
      "MagFilter": "J",
      "association_type": "SN",
      "catalogue_object_id": "03391748-2710138",
      "catalogue_object_type": "galaxy",


## All object data

In [10]:
result = L.object(objectId, lasair_added=True)
print(json.dumps(result, indent=2))

{
  "diaObjectId": "3521874258609307650",
  "lasairData": {
    "nSources": 7,
    "firstDiaSourceMJD": 999.0,
    "lastDiaSourceMJD": 61336.20732022106,
    "rasex": "03:39:16.936",
    "decsex": "-27:10:34.646",
    "ec_lon": 43.44492858477278,
    "ec_lat": -45.094607447840765,
    "now_mjd": "60795.45",
    "mjdmin_ago": 59796.44865981098,
    "mjdmax_ago": -540.7586604100798,
    "discMjd": 61314.27337522106,
    "discUtc": "2026-10-01 06:33:39",
    "discMag": "4045.03\u00b1961.56",
    "discFilter": "g",
    "latestMjd": 61336.20732022106,
    "latestUtc": "2026-10-23 04:58:32",
    "latestMag": "-751.77\u00b1928.96",
    "latestFilter": "g",
    "peakMjd": 61336.20732022106,
    "peakUtc": "2026-10-23 04:58:32",
    "peakMag": "-751.77\u00b1928.96",
    "peakFilter": "g",
    "sherlock": {
      "diaObjectId": 3521874258609307650,
      "classification": "SN",
      "association_type": "SN",
      "catalogue_table_name": "2MASS",
      "catalogue_object_id": "03391748-2710138",

In [11]:
result = L.object(objectId, lasair_added=False)
print15(json.dumps(result, indent=2))

{
  "diaObjectId": "3521874258609307650",
  "diaSources": [
    {
      "diaSourceId": 3527246281296576548,
      "midpointMjdTai": 61314.27337522106,
      "band": "g",
      "psfFlux": 4045.029296875,
      "psfFluxErr": 961.5590209960938
    },
    {
      "diaSourceId": 3527246818167488548,
      "midpointMjdTai": 61314.27337522106,
      "band": "g",
      "psfFlux": 4044.686279296875,
